In [60]:
import catboost
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import ensemble

import pandas as pd
import numpy as np

In [61]:
pd.options.display.max_columns = None
df = pd.read_csv('../data/hp/train.csv', index_col='Id')
df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,Wood,Gd,TA,No,GLQ,732,Unf,0,64,796,GasA,Ex,Y,SBrkr,796,566,0,1362,1,0,1,1,1,1,TA,5,Typ,0,NaN,Attchd,1993.0,Unf,2,480,TA,TA,Y,40,30,0,320,0,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186.0,Gd,TA,PConc,Ex,TA,Av,GLQ,1369,Unf,0,317,1686,GasA,Ex,Y,SBrkr,1694,0,0,1694,1,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2004.0,RFn,2,636,TA,TA,Y,255,57,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,1973,1973,Gable,CompShg,HdBoard,HdBoard,Stone,240.0,TA,TA,CBlock,Gd,TA,Mn,ALQ,859,BLQ,32,216,1107,GasA,Ex,Y,SBrkr,1107,983,0,2090,1,0,2,1,3,1,TA,7,Typ,2,TA,Attchd,1973.0,RFn,2,484,TA,TA,Y,235,204,228,0,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,CompShg,BrkFace,Wd Shng,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,952,952,GasA,Gd,Y,FuseF,1022,752,0,1774,0,0,2,0,2,2,TA,8,Min1,2,TA,Detchd,1931.0,Unf,2,468,Fa,TA,Y,90,0,205,0,0,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900


In [62]:
df["LotFrontage"].fillna(value=0.0, inplace=True)
df["GarageYrBlt"].fillna(value=0.0, inplace=True)
df["MasVnrType"].fillna(value="0", inplace=True)
df["MasVnrArea"].fillna(value=0.0, inplace=True)
df["BsmtQual"].fillna(value="0", inplace=True)

df["BsmtCond"].fillna(value="0", inplace=True)
df["BsmtExposure"].fillna(value="0", inplace=True)
df["BsmtFinType1"].fillna(value="0", inplace=True)
df["BsmtFinType2"].fillna(value="0", inplace=True)
df["Electrical"].fillna(value="0", inplace=True)
df["FireplaceQu"].fillna(value="0", inplace=True)
df["GarageType"].fillna(value="0", inplace=True)
df["GarageFinish"].fillna(value="0", inplace=True)
df["GarageQual"].fillna(value="0", inplace=True)
df["GarageCond"].fillna(value="0", inplace=True)
df["Fence"].fillna(value="0", inplace=True)

In [63]:
# df["Alley"].fillna(value = '0', inplace = True)
# df["PoolQC"].fillna(value = '0', inplace = True)
del df["Alley"] 
del df["PoolQC"]
del df["MiscFeature"]
# delete PoolQC and MiscFeature because they are redundant, possibly delete alley because almost all is NaN

In [64]:
CategoricalDataCols = ["LotShape", "LandContour", "Utilities", "LotConfig", "LandSlope", "Neighborhood", "Condition1", 
                       "Condition2", "BldgType", "HouseStyle", "RoofStyle", "RoofMatl", "Exterior1st", "Exterior2nd", 
                       "MasVnrType", "MasVnrArea", "ExterQual", "ExterCond", "Foundation", "BsmtQual", 
                       "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "Heating", "HeatingQC", 
                       "CentralAir", "Electrical", "KitchenQual", "Functional", "FireplaceQu", "GarageType", 
                       "GarageFinish", "GarageQual", "GarageCond", "PavedDrive", "Fence", "SaleType", 
                       "SaleCondition", "Street", "MSZoning", "MSSubClass"]

In [65]:
Labels = df["SalePrice"]
Data = df.drop("SalePrice", axis=1)


In [66]:
for col in CategoricalDataCols:
    Data[col] = pd.Categorical(Data[col])

train_data, test_data, train_labels, test_labels = train_test_split(Data, Labels, random_state=98987, test_size=0.25)

In [67]:
train_data_cats = train_data[CategoricalDataCols]
test_data_cats = test_data[CategoricalDataCols]
oh_train_data = train_data
for col in CategoricalDataCols:
    oh_train_data = oh_train_data.drop(col, axis=1)
oh_test_data = test_data
for col in CategoricalDataCols:
    oh_test_data = oh_test_data.drop(col, axis=1)

display(oh_test_data.head())

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
883,0.0,9636,6,5,1992,1993,0,0,808,808,808,785,0,1593,0,0,2,1,3,1,7,1,1993.0,2,389,342,40,0,0,0,0,0,12,2009
1443,85.0,11003,10,5,2008,2008,765,0,252,1017,1026,981,0,2007,1,0,2,1,3,1,10,1,2008.0,3,812,168,52,0,0,0,0,0,4,2009
1136,60.0,6180,6,5,1926,1950,0,0,960,960,986,0,0,986,0,0,1,0,2,1,5,1,1926.0,1,180,0,128,0,0,0,0,0,5,2007
631,50.0,9000,5,6,1880,1991,0,0,636,636,1089,661,0,1750,0,0,1,0,3,1,8,0,1937.0,1,240,0,0,293,0,0,0,0,6,2006
1067,59.0,7837,6,7,1993,1994,0,0,799,799,799,772,0,1571,0,0,2,1,3,1,7,1,1993.0,2,380,0,40,0,0,0,0,0,5,2009


In [68]:
one_hot = sklearn.preprocessing.OneHotEncoder(handle_unknown='ignore')
train_data_cats = one_hot.fit_transform(train_data_cats).toarray()

In [69]:
test_data_cats = one_hot.transform(test_data_cats).toarray()

In [70]:
for i in range(len(train_data_cats[0])):
#     print(len(train_data_cats))
#     print(len(train_data_cats[0]))
#     print
#     print(i)
    oh_train_data['one-hot' + str(i)] = train_data_cats[:, i]
    
for i in range(len(test_data_cats[0])):
    oh_test_data['one-hot' + str(i)] = test_data_cats[:, i]

display(oh_train_data.head())
display(oh_train_data.head())

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,one-hot0,one-hot1,one-hot2,one-hot3,one-hot4,one-hot5,one-hot6,one-hot7,one-hot8,one-hot9,one-hot10,one-hot11,one-hot12,one-hot13,one-hot14,one-hot15,one-hot16,one-hot17,one-hot18,one-hot19,one-hot20,one-hot21,one-hot22,one-hot23,one-hot24,one-hot25,one-hot26,one-hot27,one-hot28,one-hot29,one-hot30,one-hot31,one-hot32,one-hot33,one-hot34,one-hot35,one-hot36,one-hot37,one-hot38,one-hot39,one-hot40,one-hot41,one-hot42,one-hot43,one-hot44,one-hot45,one-hot46,one-hot47,one-hot48,one-hot49,one-hot50,one-hot51,one-hot52,one-hot53,one-hot54,one-hot55,one-hot56,one-hot57,one-hot58,one-hot59,one-hot60,one-hot61,one-hot62,one-hot63,one-hot64,one-hot65,one-hot66,one-hot67,one-hot68,one-hot69,one-hot70,one-hot71,one-hot72,one-hot73,one-hot74,one-hot75,one-hot76,one-hot77,one-hot78,one-hot79,one-hot80,one-hot81,one-hot82,one-hot83,one-hot84,one-hot85,one-hot86,one-hot87,one-hot88,one-hot89,one-hot90,one-hot91,one-hot92,one-hot93,one-hot94,one-hot95,one-hot96,one-hot97,one-hot98,one-hot99,one-hot100,one-hot101,one-hot102,one-hot103,one-hot104,one-hot105,one-hot106,one-hot107,one-hot108,one-hot109,one-hot110,one-hot111,one-hot112,one-hot113,one-hot114,one-hot115,one-hot116,one-hot117,one-hot118,one-hot119,one-hot120,one-hot121,one-hot122,one-hot123,one-hot124,one-hot125,one-hot126,one-hot127,one-hot128,one-hot129,one-hot130,one-hot131,one-hot132,one-hot133,one-hot134,one-hot135,one-hot136,one-hot137,one-hot138,one-hot139,one-hot140,one-hot141,one-hot142,one-hot143,one-hot144,one-hot145,one-hot146,one-hot147,one-hot148,one-hot149,one-hot150,one-hot151,one-hot152,one-hot153,one-hot154,one-hot155,one-hot156,one-hot157,one-hot158,one-hot159,one-hot160,one-hot161,one-hot162,one-hot163,one-hot164,one-hot165,one-hot166,one-hot167,one-hot168,one-hot169,one-hot170,one-hot171,one-hot172,one-hot173,one-hot174,one-hot175,one-hot176,one-hot177,one-hot178,one-hot179,one-hot180,one-hot181,one-hot182,one-hot183,one-hot184,one-hot185,one-hot186,one-hot187,one-hot188,one-hot189,one-hot190,one-hot191,one-hot192,one-hot193,one-hot194,one-hot195,one-hot196,one-hot197,one-hot198,one-hot199,one-hot200,one-hot201,one-hot202,one-hot203,one-hot204,one-hot205,one-hot206,one-hot207,one-hot208,one-hot209,one-hot210,one-hot211,one-hot212,one-hot213,one-hot214,one-hot215,one-hot216,one-hot217,one-hot218,one-hot219,one-hot220,one-hot221,one-hot222,one-hot223,one-hot224,one-hot225,one-hot226,one-hot227,one-hot228,one-hot229,one-hot230,one-hot231,one-hot232,one-hot233,one-hot234,one-hot235,one-hot236,one-hot237,one-hot238,one-hot239,one-hot240,one-hot241,one-hot242,one-hot243,one-hot244,one-hot245,one-hot246,one-hot247,one-hot248,one-hot249,one-hot250,one-hot251,one-hot252,one-hot253,one-hot254,one-hot255,one-hot256,one-hot257,one-hot258,one-hot259,one-hot260,one-hot261,one-hot262,one-hot263,one-hot264,one-hot265,one-hot266,one-hot267,one-hot268,one-hot269,one-hot270,one-hot271,one-hot272,one-hot273,one-hot274,one-hot275,one-hot276,one-hot277,one-hot278,one-hot279,one-hot280,one-hot281,one-hot282,one-hot283,one-hot284,one-hot285,one-hot286,one-hot287,one-hot288,one-hot289,one-hot290,one-hot291,one-hot292,one-hot293,one-hot294,one-hot295,one-hot296,one-hot297,one-hot298,one-hot299,one-hot300,one-hot301,one-hot302,one-hot303,one-hot304,one-hot305,one-hot306,one-hot307,one-hot308,one-hot309,one-hot310,one-hot311,one-hot312,one-hot313,one-hot314,one-hot315,one-hot316,one-hot317,one-hot318,one-hot319,one-hot320,one-hot321,one-hot322,one-hot323,one-hot324,one-hot325,one-hot326,one-hot327,one-hot328,one-hot329,one-hot330,one-hot331,one-hot332,one-hot333,one-hot334,one-hot335,one-hot336,one-hot337,one-hot338,one-hot339

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,one-hot0,one-hot1,one-hot2,one-hot3,one-hot4,one-hot5,one-hot6,one-hot7,one-hot8,one-hot9,one-hot10,one-hot11,one-hot12,one-hot13,one-hot14,one-hot15,one-hot16,one-hot17,one-hot18,one-hot19,one-hot20,one-hot21,one-hot22,one-hot23,one-hot24,one-hot25,one-hot26,one-hot27,one-hot28,one-hot29,one-hot30,one-hot31,one-hot32,one-hot33,one-hot34,one-hot35,one-hot36,one-hot37,one-hot38,one-hot39,one-hot40,one-hot41,one-hot42,one-hot43,one-hot44,one-hot45,one-hot46,one-hot47,one-hot48,one-hot49,one-hot50,one-hot51,one-hot52,one-hot53,one-hot54,one-hot55,one-hot56,one-hot57,one-hot58,one-hot59,one-hot60,one-hot61,one-hot62,one-hot63,one-hot64,one-hot65,one-hot66,one-hot67,one-hot68,one-hot69,one-hot70,one-hot71,one-hot72,one-hot73,one-hot74,one-hot75,one-hot76,one-hot77,one-hot78,one-hot79,one-hot80,one-hot81,one-hot82,one-hot83,one-hot84,one-hot85,one-hot86,one-hot87,one-hot88,one-hot89,one-hot90,one-hot91,one-hot92,one-hot93,one-hot94,one-hot95,one-hot96,one-hot97,one-hot98,one-hot99,one-hot100,one-hot101,one-hot102,one-hot103,one-hot104,one-hot105,one-hot106,one-hot107,one-hot108,one-hot109,one-hot110,one-hot111,one-hot112,one-hot113,one-hot114,one-hot115,one-hot116,one-hot117,one-hot118,one-hot119,one-hot120,one-hot121,one-hot122,one-hot123,one-hot124,one-hot125,one-hot126,one-hot127,one-hot128,one-hot129,one-hot130,one-hot131,one-hot132,one-hot133,one-hot134,one-hot135,one-hot136,one-hot137,one-hot138,one-hot139,one-hot140,one-hot141,one-hot142,one-hot143,one-hot144,one-hot145,one-hot146,one-hot147,one-hot148,one-hot149,one-hot150,one-hot151,one-hot152,one-hot153,one-hot154,one-hot155,one-hot156,one-hot157,one-hot158,one-hot159,one-hot160,one-hot161,one-hot162,one-hot163,one-hot164,one-hot165,one-hot166,one-hot167,one-hot168,one-hot169,one-hot170,one-hot171,one-hot172,one-hot173,one-hot174,one-hot175,one-hot176,one-hot177,one-hot178,one-hot179,one-hot180,one-hot181,one-hot182,one-hot183,one-hot184,one-hot185,one-hot186,one-hot187,one-hot188,one-hot189,one-hot190,one-hot191,one-hot192,one-hot193,one-hot194,one-hot195,one-hot196,one-hot197,one-hot198,one-hot199,one-hot200,one-hot201,one-hot202,one-hot203,one-hot204,one-hot205,one-hot206,one-hot207,one-hot208,one-hot209,one-hot210,one-hot211,one-hot212,one-hot213,one-hot214,one-hot215,one-hot216,one-hot217,one-hot218,one-hot219,one-hot220,one-hot221,one-hot222,one-hot223,one-hot224,one-hot225,one-hot226,one-hot227,one-hot228,one-hot229,one-hot230,one-hot231,one-hot232,one-hot233,one-hot234,one-hot235,one-hot236,one-hot237,one-hot238,one-hot239,one-hot240,one-hot241,one-hot242,one-hot243,one-hot244,one-hot245,one-hot246,one-hot247,one-hot248,one-hot249,one-hot250,one-hot251,one-hot252,one-hot253,one-hot254,one-hot255,one-hot256,one-hot257,one-hot258,one-hot259,one-hot260,one-hot261,one-hot262,one-hot263,one-hot264,one-hot265,one-hot266,one-hot267,one-hot268,one-hot269,one-hot270,one-hot271,one-hot272,one-hot273,one-hot274,one-hot275,one-hot276,one-hot277,one-hot278,one-hot279,one-hot280,one-hot281,one-hot282,one-hot283,one-hot284,one-hot285,one-hot286,one-hot287,one-hot288,one-hot289,one-hot290,one-hot291,one-hot292,one-hot293,one-hot294,one-hot295,one-hot296,one-hot297,one-hot298,one-hot299,one-hot300,one-hot301,one-hot302,one-hot303,one-hot304,one-hot305,one-hot306,one-hot307,one-hot308,one-hot309,one-hot310,one-hot311,one-hot312,one-hot313,one-hot314,one-hot315,one-hot316,one-hot317,one-hot318,one-hot319,one-hot320,one-hot321,one-hot322,one-hot323,one-hot324,one-hot325,one-hot326,one-hot327,one-hot328,one-hot329,one-hot330,one-hot331,one-hot332,one-hot333,one-hot334,one-hot335,one-hot336,one-hot337,one-hot338,one-hot339

In [71]:
sklearn_boost = ensemble.GradientBoostingRegressor(loss='ls', learning_rate=0.1, n_estimators=100, 
                                   subsample=1.0, criterion='friedman_mse', min_samples_split=2, 
                                   min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, 
                                   min_impurity_decrease=0.0, min_impurity_split=None, init=None, 
                                   random_state=98987, max_features=None, alpha=0.9, verbose=0, 
                                   max_leaf_nodes=None, warm_start=False, presort='deprecated', 
                                   validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)

sklearn_boost.fit(oh_train_data, train_labels)
print("train score", sklearn_boost.score(oh_train_data, train_labels))
print("test score", sklearn_boost.score(oh_test_data, test_labels))

train score 0.9725603672039187
test score 0.8420232319410241


In [72]:
prediction = sklearn_boost.predict(oh_test_data)

In [73]:
abs_errors = abs(prediction - test_labels.to_numpy())

In [74]:
comparison = np.zeros([prediction.size, 2])
for i in range(prediction.size):
    comparison[i][0] = prediction[i]
    comparison[i][1] = test_labels.to_numpy()[i]

In [75]:
abs_errors = abs(prediction - test_labels.to_numpy())
mae = abs_errors.sum() / len(abs_errors)
print(mae)

17580.600243838977


In [76]:
print(1 - mae / (test_labels.to_numpy().sum() / test_labels.to_numpy().size))

0.9016843837869448


In [77]:
print(comparison)

[[165103.27738684 178000.        ]
 [365274.74605032 310000.        ]
 [107374.08474568 102000.        ]
 [132254.59957051 124000.        ]
 [165925.31283189 178000.        ]
 [144171.2322821  156000.        ]
 [237089.13286699 272000.        ]
 [170364.80964282 184000.        ]
 [228577.76831732 222000.        ]
 [119348.50786084 125500.        ]
 [209197.29250285 231500.        ]
 [244497.42198148 268000.        ]
 [ 87877.80690423  75000.        ]
 [390373.30721374 335000.        ]
 [126383.22437679 130250.        ]
 [177752.19924505 190000.        ]
 [158046.08245377 171900.        ]
 [170673.34873497 160000.        ]
 [304839.13643125 348000.        ]
 [173635.01272762 197500.        ]
 [364662.19532476 276000.        ]
 [129949.86902853 111250.        ]
 [136157.35465049 144000.        ]
 [146902.69944461 146800.        ]
 [355322.18300659 319000.        ]
 [131542.50482171  95000.        ]
 [165259.26755598 165150.        ]
 [253000.16981595 235000.        ]
 [133017.99653781 15